### Import thư viện cần thiết

In [1]:
import requests
import datetime
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

### Tạo Webdriver

In [2]:
apart_arr = []
# Declare browser
options = Options()
options.add_argument("--headless")
options.add_argument("---window-size=1920,1080")
PATH = "D:\Tai_Phan_Mem\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(PATH)#, options = options)

C:\Users\Admin\AppData\Local\Temp\ipykernel_22848\3964112724.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)#, options = options)


### Khởi tạo Dataframe chứa dữ liệu thu thập

In [3]:
# DATAFRAME FEATURES
# address, price, area, property_status, price_per_m2, bedroom, toilets, floornumber, property_legal, furniture
df_columns = ['address', 'area', 'price', 'property_status', 'price_per_m2', 'bedroom', 'toilets', 'floornumber','property_legal', 'furniture']
len(df_columns)
# Khởi tạo dataframe rỗng, tạo sẵn các cột
df = pd.DataFrame(columns = df_columns)
df.drop(df.index,inplace=True) 
df.shape

(0, 10)

### 1. Lấy link (đường dẫn) đến các chung cư 

In [4]:
# hàm lấy link của các trang chủ (num_page = số trang)
# Thêm vào
def get_apart_links(url, num_page):
    apartments_link = []
    for i in range(num_page):
        driver2 = webdriver.Chrome(PATH) #, options = options)
        driver2.get(url + str(i))
        
        # Open URL
        soup = BeautifulSoup(driver2.page_source)
        a = soup.find_all("a",class_='AdItem_adItem__USIDA')
        for i in range(len(a)):
            link ="https://nha.chotot.com/"+a[i]['href']
            apartments_link.append(link)
    return apartments_link


In [5]:
url = "https://www.nhatot.com/mua-ban-can-ho-chung-cu-tp-ho-chi-minh?page="
num_page = 55
apartment_links = get_apart_links(url, num_page)
print('Số lượng chung cư sẽ lấy thông tin:', len(apartment_links))

C:\Users\Admin\AppData\Local\Temp\ipykernel_22848\2369627906.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome(PATH) #, options = options)


Số lượng chung cư sẽ lấy thông tin: 1115


### 2. Truy cập từng chung cư để lấy dữ liệu

In [6]:
# Trả về thông tin (list) của 1 apartment
def get_info_of_apartment(apartment_link):
    driver1 = webdriver.Chrome(PATH) #, options = options)
    driver1.get(apartment_link)
    driver1.execute_script('window.scrollTo(0,700)')

    try:
        xemthem = driver1.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div[1]/div/div[4]/div/div[3]/button')
        xemthem.click()
    except: 
        pass
    soup1 = BeautifulSoup(driver1.page_source)
    
    price = soup1.find('span', itemprop = 'price')
    price = price.text.split(' - ')[0]
    area = soup1.find('span', itemprop = 'size')
    address = soup1.find(class_ = 'fz13')
    property_status = soup1.find('span', itemprop = 'property_status')
    price_per_m2 = soup1.find('span', itemprop="price_m2")
    bedroom = soup1.find('span', itemprop="rooms")
    toilets = soup1.find('span', itemprop="toilets")
    floornumber = soup1.find('span', itemprop="floornumber")
    property_legal = soup1.find('span', itemprop="property_legal_document")
    furniture = soup1.find('span', itemprop = 'furnishing_sell')
    
    info_list = [] # HTML type
    info_text = [] # text
    info_list.extend([address, price, area, property_status, price_per_m2, bedroom, toilets, floornumber, property_legal, furniture])
    info_text.extend([address.text[:-10], price])

    for feature in info_list[2:]:
        if feature != None:
            info_text.append(feature.text)
        else:
            info_text.append('')
    driver1.quit()
    return info_text
    


In [7]:
# Lấy thông tin từng trang, thêm vào df

def get_info_of_page(apart_arr):
    df_temp = df.copy()
    for apart in apart_arr:
        apart_info = get_info_of_apartment(apart)
        df_temp.loc[len(df)] = apart_info
    return df_temp
print(len(apartment_links))
# get_info_of_page(apartment_links)        

1115


In [8]:
df1 = get_info_of_page(apartment_links[0:100])
df1.to_csv('chotot-data-1.csv', index = False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_22848\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)


In [9]:
df2 = get_info_of_page(apartment_links[100:200])
df2.to_csv('chotot-data-2.csv', index = False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_22848\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)


In [ ]:
df3 = get_info_of_page(apartment_links[200:300])
df3.to_csv('chotot-data-3.csv', index = False)

In [ ]:
df4 = get_info_of_page(apartment_links[300:400])
df4.to_csv('chotot-data-4.csv', index = False)

In [ ]:
df5 = get_info_of_page(apartment_links[400:500])
df5.to_csv('chotot-data-5.csv', index = False)

In [ ]:
df6 = get_info_of_page(apartment_links[500:600])
df6.to_csv('chotot-data-6.csv', index = False)

In [ ]:
df7 = get_info_of_page(apartment_links[600:700])
df7.to_csv('chotot-data-7.csv', index = False)


In [ ]:
df8 = get_info_of_page(apartment_links[700:800])
df8.to_csv('chotot-data-8.csv', index = False)

In [ ]:
df9 = get_info_of_page(apartment_links[800:900])
df9.to_csv('chotot-data-9.csv', index = False)

In [ ]:
df10 = get_info_of_page(apartment_links[900:1000])
df10.to_csv('chotot-data-10.csv', index = False)

In [ ]:
df11 = get_info_of_page(apartment_links[1000:])
df11.to_csv('chotot-data-11.csv', index = False)

In [14]:
df

,address,area,price,property_status,price_per_m2,bedroom,toilets,floornumber,property_legal,furniture


In [ ]:
# Xuất file csv
df.to_csv('chotot-data-1.csv', index = False)